In [8]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

In [9]:
data = gpd.read_file('./tl_2010_55079_tract10/tl_2010_55079_tract10.shp')
data = data.sort_values('TRACTCE10')
data['White'] = 0
data['Black or African American'] = 0 
data['American Indian and Ala Native'] = 0
data['Asian'] = 0
data['Native Hawaiian/other Pac Isl'] = 0
data['Multiple Race'] = 0
data['Other Race'] = 0

In [10]:
df = pd.read_csv('DEC_10_SF1_QTP3_with_ann.csv')
census_drop_list = ['HD01_S01','HD01_S02','HD02_S01','HD02_S02','HD02_S03',
                   'HD02_S04','HD02_S05','HD02_S06','HD02_S07','HD02_S08',
                   'HD02_S09','HD02_S10','HD02_S18','HD02_S23','HD02_S24',
                   'HD02_S25','HD02_S26','HD02_S27','HD02_S28','HD01_S06',
                   'HD01_S07','HD01_S08','HD01_S09','HD01_S25','HD01_S26',
                   'HD01_S27','HD01_S28','HD02_S29','HD02_S30','HD02_S31',
                   'HD01_S31','HD01_S32','HD02_S32','HD01_S33','HD02_S33',
                   'HD01_S34','HD02_S34','HD01_S35','HD02_S35','HD01_S29',
                   'HD01_S30','HD02_S36','HD01_S37','HD02_S37','HD02_S38',
                   'HD01_S39','HD02_S39','HD01_S40','HD02_S40','HD01_S41',
                   'HD02_S41','HD01_S42','HD02_S42']

df =df.drop(census_drop_list,axis=1)

df.columns = df.loc[0]
df = df.drop(0,axis=0)

In [11]:
for item in df.Id2:
    series = df[df['Id2'] == item].drop('Id',axis =1).drop(
    'Id2', axis=1).drop('Geography', axis=1).drop(
    'Number; RACE AND HISPANIC OR LATINO - Total population',axis=1).drop(
    'Number; RACE AND HISPANIC OR LATINO - Total population - One race - Hispanic or Latino',axis=1)
    
    data.loc[data.GEOID10 == item,['White','Black or African American',
                                   'American Indian and Ala Native',
                                   'Asian', 'Native Hawaiian/other Pac Isl',
                                   'Multiple Race', 'Other Race']] = series.sum(axis=0).astype(np.int).tolist()
    

In [12]:
missing_list = []
for index in data.index:
    if np.all(data.loc[index,['White','Black or African American','Asian',
            'Native Hawaiian/other Pac Isl','Multiple Race','Other Race']] == 0):
        missing_list.append(data.loc[index,'TRACTCE10'])
  #      data = data.drop(index)
  #  else:
        #data = data.drop(index)
        
missing_list.sort()
missing_list

['980000', '990000']

In [13]:
delete_list = ['050101','050102','040100','060102','060200','070200',
              '070100','080100','035200','030100','035100','070300',
              '080400','990000','185100','180100','180200','180500',
              '170200','170300','170100','170700','170600','170300',
              '160100','150100','120501','140202','150304','060101',
              '080200','185200','180300','180400','120502','120400',
              '120202','140201','140100','150301','130200','130100',
              '120101','120102','101000','101100','100900','090100',
              '090200','091000','090900','091100','091200','091400',
              '091300','090700','185300','090300','090600','100800',
              '110100','100100','100200','101600','100300','100400',
              '101500','101700','101800','101200','101300','101400',
              '100500','100600','100700','120201','120203','150303',
              '090800','120300','080300']
data = data.drop(data.loc[data.TRACTCE10.isin(delete_list)].index)

In [14]:
try:
    os.remove('../KMeans/zipcodes.js')
except FileNotFoundError:
    pass
data.to_file('../KMeans/census.geoJSON', driver='GeoJSON')
data.to_file('../KMeans/zipcodes.js', driver='GeoJSON')

with open('../KMeans/zipcodes.js', 'r') as original: data = original.read()
with open('../KMeans/zipcodes.js', 'w') as modified: modified.write("var zipcodes =" 
                                                        + data +';')